In [98]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import math

In [ ]:
def log_df(df, limit=5):
    print(df.shape)
    return df.head(limit)

int_dtype = {'playlist_position': 'Int64', 'price': 'Int64',
             'serial_number':'Int64', 'product_id': 'Int64'}


In [188]:
def fetch_content(url):
    # print(url)
    page = requests.get(url)
    return BeautifulSoup(page.content, "html.parser")


def calculate_page_count(htmlContent):
    count_container = htmlContent.find("span", class_="toolbar-number")

    if count_container:
        count = int(count_container.text)
        return math.ceil(count/24)
    else:
        return 0


In [268]:
def process_title(title):
    matches = re.findall("\\(.*?([LX][A-Za-z0-9- ]+)\\)", title)
    if matches[0]:
        parts = matches[0].split(' ')
        model = parts[0]
        serial_match = re.search("[0-9]+", parts[-1])
        serial_number = int(serial_match[0])

        return {"model": model, 'serial_number':serial_number}

def process_list_content(htmlContent):
    units = []

    items = htmlContent.find_all("div", class_="product-item-details")
    for item in items:
        link = item.find('a', class_='product-item-link')
        href = link['href']
        title = link.text.strip()

        title_data = process_title(title)

        price_container = item.find('span', class_='price-wrapper')
        price = price_container['data-price-amount']

        details_content = fetch_content(href)
        details_data = process_details_content(details_content)

        id_container = item.find("div", class_="price-final_price")
        product_id = id_container["data-product-id"]

        units.append({
            "listing_title": title,
            "listing_url": href,
            "model": title_data['model'],
            "serial_number": title_data['serial_number'],
            "price": int(price),
            "video_id": details_data['video_id'],
            "product_id": int(product_id)
        })

    return units

def process_details_content(htmlContent):
    embed_container = htmlContent.find('div', class_ = 'embed-container')
    iframe = embed_container.find('iframe')
    parts = iframe['src'].split('/')
    params = parts[-1].split('?')

    id_container = htmlContent.find("div", class_="price-final_price")
    product_id = id_container["data-product-id"]


    return {'video_id': params[0], "product_id": int(product_id)}


In [185]:
OLI_URL = "https://theukulelesite.com/shop-by/brand/oli.html"
html_countent = fetch_content(OLI_URL)



https://theukulelesite.com/shop-by/brand/oli.html


In [ ]:

units = []

units = process_list_content(html_countent)

In [189]:
page_count = calculate_page_count(html_countent)

for count in range(page_count):
    if count == 0:
        continue

    page_url = f'{OLI_URL}?p={count + 1}'
    html_countent = fetch_content(page_url)
    units = units + process_list_content(html_countent)



In [191]:
df = pd.DataFrame(units)
log_df(df)

(62, 6)


,listing_title,listing_url,model,serial_number,price,video_id
0,'Oli Redwood Rosewood Concert (X2-CRR-SE 1665),https://theukulelesite.com/shop-by/brand/oli/o...,X2-CRR-SE,1665,1395,P8TOa1_NZxI
1,'Oli Port Orford Cedar Myrtle Concert Gloss (L...,https://theukulelesite.com/shop-by/brand/oli/o...,L1SP-CCPMR,1542,895,gsqmlrAh3m4
2,'Oli Myrtle Concert Gloss (L1-CMR 1570),https://theukulelesite.com/shop-by/brand/oli/o...,L1-CMR,1570,795,WGt5K3woqho
3,'Oli Redwood Rosewood Tenor (X2-TRR-SE 1690),https://theukulelesite.com/shop-by/brand/oli/o...,X2-TRR-SE,1690,1495,ecDhE4DQ0jk
4,'Oli Sinker Cypress Quilted Maple LE Concert (...,https://theukulelesite.com/shop-by/brand/oli/o...,L2-CSCQM-BK,1750,1395,jc6vNNrlYnk


In [192]:
df.to_csv('../raw_data/oli_listings.csv', index=False)

## testing functions

In [195]:
titles = [
    'Oli Redwood Rosewood Concert (X2-CRR-SE 1665)',
    'Oli Redwood Rosewood Concert (2024 X2-CRR-SE 1665)',
    'Oli Redwood Rosewood Concert (X2-CRR-SE 1665B)',
    'Oli Redwood Rosewood Concert (X2-CRR-SE Moon Bay 1665)'
]

for title in titles:
    print(process_title(title))

{'model': 'X2-CRR-SE', 'serial_number': 1665}
{'model': 'X2-CRR-SE', 'serial_number': 1665}
{'model': 'X2-CRR-SE', 'serial_number': 1665}
{'model': 'X2-CRR-SE', 'serial_number': 1665}


In [ ]:
from datetime import datetime

current_dateTime = datetime.now()

# 2024-02-28T22:45:18Z

date_time = current_dateTime.strftime("%Y-%m-%d %H:%M:%S")
print(date_time)

2025-10-03 17:34:57


## find sold listings

In [ ]:
df = pd.read_csv('../raw_data/oli_listings.csv', dtype=int_dtype)
log_df(df)

(71, 10)


,listing_title,listing_url,model,serial_number,price,video_id,date_added,date_sold,serial_number_y,product_id
0,'Oli Redwood Rosewood Concert (X2-CRR-SE 1665),https://theukulelesite.com/shop-by/brand/oli/o...,X2-CRR-SE,1665,1395,P8TOa1_NZxI,2025-10-03 12:00:00,2025-10-06 14:40:04,NaN,NaN
1,'Oli Port Orford Cedar Myrtle Concert Gloss (L...,https://theukulelesite.com/shop-by/brand/oli/o...,L1SP-CCPMR,1542,895,gsqmlrAh3m4,2025-10-03 12:00:00,2025-10-06 14:40:04,NaN,NaN
2,'Oli Myrtle Concert Gloss (L1-CMR 1570),https://theukulelesite.com/shop-by/brand/oli/o...,L1-CMR,1570,795,WGt5K3woqho,2025-10-03 12:00:00,NaN,NaN,NaN
3,'Oli Redwood Rosewood Tenor (X2-TRR-SE 1690),https://theukulelesite.com/shop-by/brand/oli/o...,X2-TRR-SE,1690,1495,ecDhE4DQ0jk,2025-10-03 12:00:00,2025-10-07 04:47:22,NaN,NaN
4,'Oli Sinker Cypress Quilted Maple LE Concert (...,https://theukulelesite.com/shop-by/brand/oli/o...,L2-CSCQM-BK,1750,1395,jc6vNNrlYnk,2025-10-03 12:00:00,NaN,NaN,NaN


In [204]:
site_serials = [1750]

In [212]:
temp = df[pd.isna(df['date_sold'])]

temp = temp[~temp['serial_number'].isin(site_serials)]

log_df(temp)

(58, 8)


,listing_title,listing_url,model,serial_number,price,video_id,date_added,date_sold
2,'Oli Myrtle Concert Gloss (L1-CMR 1570),https://theukulelesite.com/shop-by/brand/oli/o...,L1-CMR,1570,795,WGt5K3woqho,2025-10-03 12:00:00,NaN
3,'Oli Redwood Rosewood Tenor (X2-TRR-SE 1690),https://theukulelesite.com/shop-by/brand/oli/o...,X2-TRR-SE,1690,1495,ecDhE4DQ0jk,2025-10-03 12:00:00,NaN
5,'Oli Sinker Cypress Flame Maple Tenor (L2-TSCF...,https://theukulelesite.com/shop-by/brand/oli/o...,L2-TSCFM-BK,1675,1495,lpF2AYhpI9g,2025-10-03 12:00:00,NaN
6,'Oli Acacia Concert (X1-CA 1843),https://theukulelesite.com/shop-by/brand/oli/o...,X1-CA,1843,895,qI4e8Hm2EWk,2025-10-03 12:00:00,NaN
7,'Oli Acacia Baritone (X1-BA 1757),https://theukulelesite.com/shop-by/brand/oli/o...,X1-BA,1757,1095,RlGo3xYKAIQ,2025-10-03 12:00:00,NaN


# add product id

In [254]:
df = pd.read_csv('../raw_data/oli_listings.csv', dtype=int_dtype)
log_df(df)

(83, 9)


,listing_title,listing_url,model,serial_number,price,video_id,date_added,date_sold,product_id
0,'Oli Redwood Rosewood Concert (X2-CRR-SE 1665),https://theukulelesite.com/shop-by/brand/oli/o...,X2-CRR-SE,1665,1395,P8TOa1_NZxI,2025-10-03 12:00:00,2025-10-06 14:40:04,14256
1,'Oli Port Orford Cedar Myrtle Concert Gloss (L...,https://theukulelesite.com/shop-by/brand/oli/o...,L1SP-CCPMR,1542,895,gsqmlrAh3m4,2025-10-03 12:00:00,2025-10-06 14:40:04,<NA>
2,'Oli Myrtle Concert Gloss (L1-CMR 1570),https://theukulelesite.com/shop-by/brand/oli/o...,L1-CMR,1570,795,WGt5K3woqho,2025-10-03 12:00:00,NaN,14254
3,'Oli Redwood Rosewood Tenor (X2-TRR-SE 1690),https://theukulelesite.com/shop-by/brand/oli/o...,X2-TRR-SE,1690,1495,ecDhE4DQ0jk,2025-10-03 12:00:00,2025-10-07 04:47:22,<NA>
4,'Oli Sinker Cypress Quilted Maple LE Concert (...,https://theukulelesite.com/shop-by/brand/oli/o...,L2-CSCQM-BK,1750,1395,jc6vNNrlYnk,2025-10-03 12:00:00,2025-10-09 21:20:14,14251


In [242]:
df2 = pd.read_csv('../raw_data/draft/oli_listings_3.csv', usecols=['serial_number', 'product_id'])
log_df(df2)

(56, 2)


,serial_number,product_id
0,1572,14285
1,1608,14283
2,1790,14259
3,1777,14257
4,1570,14254


In [ ]:
foo = df.merge(df2, how='left', on='serial_number')
foo['product_id'] = foo['product_id'].astype('Int64')
log_df(foo)

(71, 9)


,listing_title,listing_url,model,serial_number,price,video_id,date_added,date_sold,product_id
0,'Oli Redwood Rosewood Concert (X2-CRR-SE 1665),https://theukulelesite.com/shop-by/brand/oli/o...,X2-CRR-SE,1665,1395,P8TOa1_NZxI,2025-10-03 12:00:00,2025-10-06 14:40:04,<NA>
1,'Oli Port Orford Cedar Myrtle Concert Gloss (L...,https://theukulelesite.com/shop-by/brand/oli/o...,L1SP-CCPMR,1542,895,gsqmlrAh3m4,2025-10-03 12:00:00,2025-10-06 14:40:04,<NA>
2,'Oli Myrtle Concert Gloss (L1-CMR 1570),https://theukulelesite.com/shop-by/brand/oli/o...,L1-CMR,1570,795,WGt5K3woqho,2025-10-03 12:00:00,NaN,14254
3,'Oli Redwood Rosewood Tenor (X2-TRR-SE 1690),https://theukulelesite.com/shop-by/brand/oli/o...,X2-TRR-SE,1690,1495,ecDhE4DQ0jk,2025-10-03 12:00:00,2025-10-07 04:47:22,<NA>
4,'Oli Sinker Cypress Quilted Maple LE Concert (...,https://theukulelesite.com/shop-by/brand/oli/o...,L2-CSCQM-BK,1750,1395,jc6vNNrlYnk,2025-10-03 12:00:00,NaN,14251


In [240]:
foo.to_csv('../raw_data/oli_listings.csv', index=False)

# add video id and product id

In [261]:
df = pd.read_csv('../raw_data/oli_listings.csv', dtype=int_dtype)
log_df(df)

(83, 9)


,listing_title,listing_url,model,serial_number,price,video_id,date_added,date_sold,product_id
0,'Oli Redwood Rosewood Concert (X2-CRR-SE 1665),https://theukulelesite.com/shop-by/brand/oli/o...,X2-CRR-SE,1665,1395,P8TOa1_NZxI,2025-10-03 12:00:00,2025-10-06 14:40:04,14256
1,'Oli Port Orford Cedar Myrtle Concert Gloss (L...,https://theukulelesite.com/shop-by/brand/oli/o...,L1SP-CCPMR,1542,895,gsqmlrAh3m4,2025-10-03 12:00:00,2025-10-06 14:40:04,<NA>
2,'Oli Myrtle Concert Gloss (L1-CMR 1570),https://theukulelesite.com/shop-by/brand/oli/o...,L1-CMR,1570,795,WGt5K3woqho,2025-10-03 12:00:00,NaN,14254
3,'Oli Redwood Rosewood Tenor (X2-TRR-SE 1690),https://theukulelesite.com/shop-by/brand/oli/o...,X2-TRR-SE,1690,1495,ecDhE4DQ0jk,2025-10-03 12:00:00,2025-10-07 04:47:22,<NA>
4,'Oli Sinker Cypress Quilted Maple LE Concert (...,https://theukulelesite.com/shop-by/brand/oli/o...,L2-CSCQM-BK,1750,1395,jc6vNNrlYnk,2025-10-03 12:00:00,2025-10-09 21:20:14,14251


In [271]:
for i, row in df.iterrows():
    if pd.isna(row['video_id']):
        details_content = fetch_content(row['listing_url'])
        details_data = process_details_content(details_content)
        df.loc[i, 'video_id'] = details_data['video_id']

    if pd.isna(row['product_id']):
        details_content = fetch_content(row['listing_url'])
        details_data = process_details_content(details_content)
        df.loc[i, 'product_id'] = details_data['product_id']


In [270]:
df.to_csv('../raw_data/oli_listings.csv', index=False)

# key moments

In [ ]:
content = fetch_content('https://www.youtube.com/watch?v=9dtlhufQk84')

In [280]:
content.find_all('a' )

[<a href="/" style="display: none;" title="YouTube"><svg height="20" id="yt-ringo2-svg" viewbox="0 0 93 20" width="93" xmlns="http://www.w3.org/2000/svg"><g><path d="M14.4848 20C14.4848 20 23.5695 20 25.8229 19.4C27.0917 19.06 28.0459 18.08 28.3808 16.87C29 14.65 29 9.98 29 9.98C29 9.98 29 5.34 28.3808 3.14C28.0459 1.9 27.0917 0.94 25.8229 0.61C23.5695 0 14.4848 0 14.4848 0C14.4848 0 5.42037 0 3.17711 0.61C1.9286 0.94 0.954148 1.9 0.59888 3.14C0 5.34 0 9.98 0 9.98C0 9.98 0 14.65 0.59888 16.87C0.954148 18.08 1.9286 19.06 3.17711 19.4C5.42037 20 14.4848 20 14.4848 20Z" fill="#FF0033"></path><path d="M19 10L11.5 5.75V14.25L19 10Z" fill="white"></path></g><g id="youtube-paths"><path d="M37.1384 18.8999V13.4399L40.6084 2.09994H38.0184L36.6984 7.24994C36.3984 8.42994 36.1284 9.65994 35.9284 10.7999H35.7684C35.6584 9.79994 35.3384 8.48994 35.0184 7.22994L33.7384 2.09994H31.1484L34.5684 13.4399V18.8999H37.1384Z"></path><path d="M44.1003 6.29994C41.0703 6.29994 40.0303 8.04994 40.0303 11.8199V1